## Analysis of an exemplary animal
3D bSSFP tumor metabolism


In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

%matplotlib widget
import ipywidgets as widgets

import datetime

import os

import hypermri
import hypermri.utils.utils_anatomical as ut_anat

import sys
%load_ext autoreload
%autoreload 2

### Load data
define path and name animal

In [ ]:
# define scan path
dirpath = 'Test_Data/metabolism_test_data/'
scans = hypermri.BrukerDir(dirpath)

In [ ]:
# load phantom data:
phantom_bssfp = scans[17]
# complex seq2d:
phantom_bssfp_img = phantom_bssfp.Load_2dseq_file(recon_num=1)

# load PHIP data:
phip_bssfp = scans[18]
# complex seq2d:
phip_bssfp_img = phip_bssfp.Load_2dseq_file(recon_num=1)

# load DNP data:
dnp_bssfp = scans[24]
# complex seq2d:
dnp_bssfp_img = dnp_bssfp.Load_2dseq_file(recon_num=1)

# load anatomical images:
coronal = scans[22]
axial = scans[21]

print("shape of seq2d = " + str(dnp_bssfp.seq2d.shape))
print("dnp_bssfp_img = " + str(dnp_bssfp_img.shape))

#### DNP ####

In [ ]:
# reshape data to match the conventional order :
# echo - read - slice - phase - reptitions - channels
_, dnp_bssfp_pv_reco = dnp_bssfp.reconstruction(seq2d = np.squeeze(dnp_bssfp_img))

print("shape of dnp_bssfp_pv_reco = " + str(dnp_bssfp_pv_reco.shape))

# reorient the data so that axial and coronal match the orienatiotn of the bSSFP data:
[_, dnp_bssfp_pv_reco, axial.seq2d, _, coronal.seq2d] = dnp_bssfp.reorient_reco(
    bssfp_seq2d=dnp_bssfp_pv_reco,
    anatomical_seq2d_ax=axial.seq2d,
    anatomical_seq2d_cor=coronal.seq2d)

print("shape of dnp_bssfp_pv_reco = " + str(dnp_bssfp_pv_reco.shape))

##### DNP - Combine multi-channel data:
Combines multichannel data. The phase of both channels is found by finding the phase that maximizes the sum of both channels.

In [ ]:
# combine multichannel data:
try:
    dnp_bssfp_pv_reco_combined = dnp_bssfp.combine_multichannel(input_data=dnp_bssfp_pv_reco)
    print("shape of dnp_bssfp_pv_reco_combined = " + str(dnp_bssfp_pv_reco_combined.shape))
except:
    dnp_bssfp_pv_reco_combined = dnp_bssfp_pv_reco

##### DNP - Shift the bSSFP data by subvoxels to match anatomical images:

In [ ]:
# shift bssfp data
dnp_bssfp_pv_reco_combined_shift = dnp_bssfp.shift_bssfp(input_data=dnp_bssfp_pv_reco_combined, # complex, reordered bSSFP data
                                 mat_bssfp=dnp_bssfp_pv_reco_combined.shape[2:4],                       # bssfp "axial" matrix size (phase and slice dim)
                                 mat_anat=axial.seq2d.shape[1:3],                                               # axial matrix size (phase and slice dim)
                                 fov_bssfp=dnp_bssfp.method['PVM_Fov'][1:3],                                    # bSSFP "axial" FOV (can be left out if bssfp and axial have same FOV)
                                 fov_anat=axial.method['PVM_Fov'],                                              # anatomical "axial" FOV (can be left out if bssfp and axial have same FOV)
                                 apply_fft=False,use_scipy_shift=True)                                                                # has to be true if bSSFP data is in image space (=False if bSSFP data is in k-space)
print("shape of dnp_bssfp_pv_reco_combined_shift = " + 
      str(dnp_bssfp_pv_reco_combined_shift.shape))

#### PHIP

In [ ]:
_, phip_bssfp_pv_reco = phip_bssfp.reconstruction(seq2d = np.squeeze(phip_bssfp_img))

# reorient the data so that axial and coronal match the orienatiotn of the bSSFP data:
[_, phip_bssfp_pv_reco,anatomical_ax, anatomical_sag, anatomical_cor] = phip_bssfp.reorient_reco(
    bssfp_seq2d=phip_bssfp_pv_reco,
    anatomical_seq2d_ax=axial.seq2d,
    anatomical_seq2d_cor=coronal.seq2d)

##### PHIP - Combine multi-channel data:

In [ ]:
# combine multichannel data:
try:
    phip_bssfp_pv_reco_combined = phip_bssfp.combine_multichannel(input_data=phip_bssfp_pv_reco)
except:
    phip_bssfp_pv_reco_combined = phip_bssfp_pv_reco

##### PHIP  - Shift the bSSFP data by subvoxels to match anatomical images:

In [ ]:
phip_bssfp_pv_reco_combined_shift = phip_bssfp.shift_bssfp(input_data=phip_bssfp_pv_reco_combined,
                                 mat_bssfp=phip_bssfp_pv_reco_combined.shape[2:4],
                                 mat_anat=axial.seq2d.shape[1:3],
                                 fov_bssfp=phip_bssfp.method['PVM_Fov'][1:3],
                                 fov_anat=axial.method['PVM_Fov'],
                                 apply_fft=False,use_scipy_shift=True) 

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, tight_layout=True, figsize=(12,5))

dnp_bssfp.plot3D_new2(bssfp_data=dnp_bssfp_pv_reco_combined_shift, # complex bSSFP data
                      coronal_image=coronal, # coronal mutlislice image object 
                      axial_image=axial,     # axial mutlislice image object
                      axlist=[ax1, ax2, ax3], # pass axes that the function will plot o
                      plot_params=None, # if you want to recreate plot, pass the json file name dnp_bssfp.load_plot_params(path_to_params="figures_shifted/fig_name.json")
                      fig=fig) # if you want a colorbar, this is necessary

## Interpolation of bssfp to anatomical resolution
To segment the datasets properly, they are interpolated to match the anatomical coronal resoltuion. To avoid interpolation artifacts, the interpolation method is set to "nearest" which means that no data is generated


In [ ]:
import time

# Interpolate DNP Data
start_time = time.time()

dnp_bssfp_itp = dnp_bssfp.interpolate_bssfp(bssfp_data=dnp_bssfp_pv_reco_combined_shift,
                            interp_size=(1,                              # echoes
                                         round(coronal.seq2d.shape[0]), # anatomical 
                                         round(coronal.seq2d.shape[1]), # anatomical
                                         coronal.seq2d.shape[2],        # slices 
                                         dnp_bssfp_pv_reco_combined_shift.shape[4],     # repetitions
                                         1),                             # channels
                            interp_method="nearest",                     # interpolation method
                            use_multiprocessing=True,                    # uses multiple cores
                            number_of_cpu_cores=None)                    # if =None and use_multiprocessing=True, automatically calcuates the nuber of CPU cores

print("--- %s seconds ---" % (time.time() - start_time))

# Interpolate PHIP Data
start_time = time.time()

phip_bssfp_itp = phip_bssfp.interpolate_bssfp(bssfp_data=phip_bssfp_pv_reco_combined_shift,
                            interp_size=(1,
                                         round(coronal.seq2d.shape[0]),
                                         round(coronal.seq2d.shape[1]),
                                         coronal.seq2d.shape[2],
                                         phip_bssfp_pv_reco_combined_shift.shape[4],
                                         1),
                            interp_method="nearest",
                            use_multiprocessing=True)
print("--- %s seconds ---" % (time.time() - start_time))

## We make a list of anatomical images to be segmented 
Define the ROI names that will be segmented.


In [ ]:
roi_names = ['bloodvessel', 'tumor', 'kidneyL','kidneyR', 'muscle','phantom', 'outside_ref','ROI1','ROI2','ROI3']

#### Segment the interesting anatomical ROIs

init list of segmenter objects

In [ ]:
segmenter_list = ut_anat.get_segmenter_list_overlayed(coronal.seq2d,
                                                       np.abs(phip_bssfp_itp),
                                                      n_rois=len(roi_names),
                                                       figsize=(6,6),
                                                      overlay=0.2,
                                                     bssfp_cmap='magma')

Draw ROIs onto the coronal anatomical images. If you misdraw you can tick the Erasing box to remove your mistakes

In [ ]:
ut_anat.draw_masks_on_anatomical(segmenter_list,roi_names)

#### Retrieve and save masks

In [ ]:
mask_dict = ut_anat.get_masks(segmenter_list,roi_keys=roi_names,plot_res=True)

#### Get the signal from the ROIs:

###### Split data into pyruvate and lactate

In [ ]:
[dnp_bssfp_itp_pyr, dnp_bssfp_itp_lac] = dnp_bssfp.split_into_pyr_lac(input_data=dnp_bssfp_itp,
                                                                      pyr_ind=0)

[phip_bssfp_itp_pyr, phip_bssfp_itp_lac] = phip_bssfp.split_into_pyr_lac(input_data=phip_bssfp_itp,
                                                                         pyr_ind=0)

# Mask data first and then integrate

In [ ]:
# init empty dictionaries
dnp_signal_pyr = {}
dnp_signal_lac = {}

phip_signal_pyr = {}
phip_signal_lac = {}

# calculate time curves of ROIs
for k in mask_dict:
    print(k)
    dnp_signal_pyr[k] =  dnp_bssfp.roi_signal_curve(input_data=np.abs(dnp_bssfp_itp_pyr),
                                                    mask_dict=mask_dict,
                                                    mask_key=k)
    
    dnp_signal_lac[k] =  dnp_bssfp.roi_signal_curve(input_data=np.abs(dnp_bssfp_itp_lac),
                                                    mask_dict=mask_dict,
                                                    mask_key=k)
    
    phip_signal_pyr[k] =  phip_bssfp.roi_signal_curve(input_data=np.abs(phip_bssfp_itp_pyr),
                                                     mask_dict=mask_dict,
                                                     mask_key=k)
    phip_signal_lac[k] =  phip_bssfp.roi_signal_curve(input_data=np.abs(phip_bssfp_itp_lac),
                                                     mask_dict=mask_dict,
                                                     mask_key=k)

##### DNP - Plot signal time curves from ROIs

In [ ]:
# get time axis:
time_axis = dnp_bssfp.calc_timeaxis()
# Define pyruvate index (0 if you started the acquisition on the pyruvate channel)
pyr_ind = 0
plt.close()
plt.figure(figsize = (12,8),tight_layout=True)
# loop through tickers and axes
for n, k in enumerate(mask_dict.keys()):
    # filter df for ticker and plot on specified axes
    ax = plt.subplot(3, 4, n + 1)
    # plot results:
    ax.plot(time_axis[pyr_ind::2],np.abs(dnp_signal_pyr[k]),label='Pyruvate')
    ax.plot(time_axis[pyr_ind+1::2],np.abs(dnp_signal_lac[k]),label='Lactate')
    # chart formatting
    ax.set_xlabel("")
    ax.legend()
    ax.set_title(k)


##### PHIP - Plot signal time curves from ROIs

In [ ]:
# get time axis:
time_axis = phip_bssfp.calc_timeaxis()
# Define pyruvate index (0 if you started the acquisition on the pyruvate channel)
pyr_ind = 0

plt.figure(figsize = (12,8),tight_layout=True)
# loop through tickers and axes
for n, k in enumerate(mask_dict.keys()):

    # filter df for ticker and plot on specified axes
    ax = plt.subplot(3, 4, n + 1)
    # plot results:
    x = time_axis[pyr_ind::2]
    y = np.abs(phip_signal_pyr[k])
    
    
    ax.plot(time_axis[pyr_ind::2],np.abs(phip_signal_pyr[k]),label='Pyruvate')
    ax.plot(time_axis[pyr_ind+1::2],np.abs(phip_signal_lac[k]),label='Lactate')
    
    # chart formatting
    ax.set_xlabel("")
    ax.legend()
    ax.set_title(k)    


#### DNP Check AUC + AUC Ratios from segmented ROIs
#### change the key string and then analyze the data

In [ ]:
print(dnp_signal_pyr.keys())

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,tight_layout=True, figsize=(12,4))
roi_key = "tumor"
dnp_bssfp.calc_AUC(signal=(np.abs(dnp_signal_pyr[roi_key]),
                           np.abs(dnp_signal_lac[roi_key])),
                           axlist = [ax1, ax2, ax3, ax4],
                           mask_key=roi_key,
                          signal_range_input=[16,89],
                          noise_range_input=[92,124],
                          noise_signal=np.abs(dnp_signal_pyr['outside_ref']),
                          apply_filter=True)


#### PHIP Check AUC + AUC Ratios from segmented ROIs

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4,tight_layout=True, figsize=(12,4))
roi_key = "tumor"
phip_bssfp.calc_AUC(signal=(np.abs(phip_signal_pyr[roi_key]),
                            np.abs(phip_signal_lac[roi_key])),
                        axlist = [ax1, ax2, ax3, ax4],
                        mask_key=roi_key,
                        signal_range_input=[7,74],
                        noise_range_input=[75,124],
                        noise_signal=np.abs(phip_signal_pyr['outside_ref']),
                        apply_filter=True,
                        display_ui=True)
